In [1]:
import pandas as pd


In [3]:
df = pd.read_csv('song_with_features_xl_format.csv',encoding='ISO-8859-1')

In [4]:
df.head(5)

,Name,Artist,Album,Popularity,Lyrics,Sentiment,Music Styles,Themes
0,Imagine - Remastered 2010,John Lennon,Imagine,79,8 ContributorsDiscographie 2021  Pop & Rock L...,NaN,NaN,NaN
1,ir,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"* """,positif,"['Rock', 'Pop', 'Rap']","['Arms', 'et', 'froid', '*', '*', '*', '*', 't...",NaN,NaN,NaN,NaN
3,A Whiter Shade Of Pale,Procol Harum,A Whiter Shade Of Pale,0,38 ContributorsA Whiter Shade of Pale Lyrics[I...,positif,[],"['Verse', 'fandango', 'cartwheels', 'floor', '..."
4,My Sweet Lord,George Harrison,All Things Must Pass (Remastered),0,58 ContributorsMy Sweet Lord Lyrics[Chorus]\nM...,positif,['Rock'],"['ContributorsMy', 'Verse', 'Chorus', 'Chorus'..."


In [5]:
df.columns


Index(['Name', 'Artist', 'Album', 'Popularity', 'Lyrics', 'Sentiment',
       'Music Styles', 'Themes'],
      dtype='object')

In [6]:
df.isnull().sum()

Name             1
Artist           8
Album           10
Popularity      10
Lyrics          32
Sentiment       54
Music Styles    54
Themes          54
dtype: int64

In [7]:
df['Lyrics'] = df['Lyrics'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [8]:
df['Lyrics'] = df['Lyrics'].str.replace(r'\[.*?\]', ' ', regex = True)

In [13]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    if pd.isna(txt):
        txt = ''
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adrie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df['Lyrics'] = df['Lyrics'].apply(lambda x: tokenization(x))

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['Lyrics'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.63452855e-03,
       5.24493556e-04, 2.00101833e-03, 2.78062193e-03, 1.50236917e-02,
       3.74330233e-03, 3.24501458e-03, 3.52685174e-03, 3.92939478e-02,
       5.21423610e-03, 9.15927747e-03, 3.12978228e-03, 1.50236917e-02,
       3.39228975e-03, 1.77080639e-03, 7.84493356e-04, 1.58414550e-02,
       1.24969926e-02, 0.00000000e+00, 2.94806721e-03, 4.42156105e-03,
       3.86471916e-03, 8.92811403e-03, 5.34003553e-03, 1.58414550e-02,
       3.56719951e-03, 0.00000000e+00, 0.00000000e+00, 1.20230140e-02,
       0.00000000e+00, 0.00000000e+00, 6.91827394e-03, 5.46277347e-03,
       7.46249492e-03, 4.31179356e-03, 5.00910808e-03, 1.58414550e-02,
       1.36741981e-02, 3.83244902e-03, 1.50236917e-02, 1.02303844e-02,
       3.83293351e-03, 3.60450022e-03, 6.04918191e-03, 5.77574190e-03,
       0.00000000e+00, 0.00000000e+00, 3.85943429e-03, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.75112501e-03, 5.54537759e-03,
      

In [22]:
df[df['Name'] == 'Imagine - Remastered 2010']

,Name,Artist,Album,Popularity,Lyrics,Sentiment,Music Styles,Themes
0,Imagine - Remastered 2010,John Lennon,Imagine,79,8 contributorsdiscographi 2021  pop & rock ly...,NaN,NaN,NaN


In [28]:
def recommendation(song_df):
    idx = df[df['Name'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]]['Name'])
        
    return songs

In [29]:
recommendation('Imagine - Remastered 2010')

['You Send Me - Remastered',
 'Princess Diana (with Nicki Minaj)',
 'Superhero (Heroes & Villains) [with Future & Chris Brown]',
 'Bellissima - Radio Edit',
 'La Vie en rose',
 'Évidemment',
 'Évidemment',
 '10:35',
 'Cupid - Twin Ver.',
 'Cupid - Twin Ver.',
 'Cupid - Twin Ver.',
 "I Don't Like It, I Love It (feat. Robin Thicke & Verdine White)",
 'Where Are Ü Now (with Justin Bieber)',
 'Tha Crossroads',
 "I'm Not Here To Make Friends",
 'Ella Baila Sola',
 'Ella Baila Sola',
 'Ella Baila Sola',
 'Ella Baila Sola',
 'Down Under (feat. Colin Hay)']

In [30]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))